### Loading models and Setting up ensembling

#### Imports

In [0]:
!pip install transformers

     |████████████████████████████████| 675kB 11.0MB/s 
     |████████████████████████████████| 1.1MB 48.7MB/s 
     |████████████████████████████████| 890kB 52.4MB/s 
     |████████████████████████████████| 3.8MB 52.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=c2c861539827e9d8aa552f31693cedbffe07be230e8929b56dd7e383d57190e2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import torch
import numpy as np
import json
from google.colab import output

#### Prediction functions

That's where the imported models go.
Calls to ensembling methods

The encode_plus() method is a shortcut and you can expect other models not to have such help. So ensembling functions may need to be edited a little.

##### Triple

In [0]:
def ensemble_triple(model_x, model_y, model_z, tokenizer_x, tokenizer_y, tokenizer_z, question, answer_text, alpha=.5):
    """
    Combines outputs from three transformers.
    NOTE: if tokenizer doesn't encode into a tuple / dict then make sure
    to unpack dictionary, input id's and input mask.
    Alpha must be smaller than .5.
    """

    # ======== Automated solution ========
    # Get token segment id's with encode plus
    max_len = 300

    input_dict_x = tokenizer_x.encode_plus(question, answer_text, max_length=max_len, pad_to_max_length=True, return_tensors='pt')
    input_dict_y = tokenizer_y.encode_plus(question, answer_text, max_length=max_len, pad_to_max_length=True, return_tensors='pt')
    input_dict_z = tokenizer_z.encode_plus(question, answer_text, max_length=max_len, pad_to_max_length=True, return_tensors='pt')

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores_x, end_scores_x = model_x(**input_dict_x)
    start_scores_y, end_scores_y = model_y(**input_dict_y)
    start_scores_z, end_scores_z = model_z(**input_dict_z)

    start = []
    start.append(torch.nn.functional.softmax(start_scores_x, dim=1).detach().numpy()[0])
    start.append(torch.nn.functional.softmax(start_scores_y, dim=1).detach().numpy()[0])
    start.append(torch.nn.functional.softmax(start_scores_z, dim=1).detach().numpy()[0])

    end = []
    end.append(torch.nn.functional.softmax(end_scores_x, dim=1).detach().numpy()[0])
    end.append(torch.nn.functional.softmax(end_scores_y, dim=1).detach().numpy()[0])
    end.append(torch.nn.functional.softmax(end_scores_z, dim=1).detach().numpy()[0])

    # model, start_ind, end_ind = ensemble_confidence_prob(start, end)
    # model, start_ind, end_ind = ensemble_confidence_score(start, end)


    # print("ANSWER: %d %d %d" %(model, start_ind, end_ind))

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    # RANKS
    # mode = {'equal', 'ranks', 'f1'}
    model, answer_start, answer_end = combine_scores2(start, end, mode="equal")
    
    # There's a chance this might hit index out of bounds
    if model == 0:
      answer = tokenizer_x.decode(input_dict_x['input_ids'][0, answer_start:answer_end+1], skip_special_tokens=True)
    elif model == 1:
      answer = tokenizer_y.decode(input_dict_y['input_ids'][0, answer_start:answer_end+1], skip_special_tokens=True)
    else:
      answer = tokenizer_z.decode(input_dict_z['input_ids'][0, answer_start:answer_end+1], skip_special_tokens=True)
    # print('Answer: "' + answer + '"')
    return answer


##### Double

In [0]:
def ensemble_double(model_x, model_y, tokenizer_x, tokenizer_y, question, answer_text, alpha=.5):
    """
    Combine two models
    """

    # ======== Automated solution ========
    # Get token segment id's with encode plus 
    ## removed arg: pad_to_max_length=True
    input_dict_x = tokenizer_x.encode_plus(question, answer_text, return_tensors='pt')
    input_dict_y = tokenizer_y.encode_plus(question, answer_text, return_tensors='pt')

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores_x, end_scores_x = model_x(**input_dict_x)
    start_scores_y, end_scores_y = model_y(**input_dict_y)

    start = []
    start.append(torch.nn.functional.softmax(start_scores_x, dim=1).detach().numpy()[0])
    start.append(torch.nn.functional.softmax(start_scores_y, dim=1).detach().numpy()[0])

    end = []
    end.append(torch.nn.functional.softmax(end_scores_x, dim=1).detach().numpy()[0])
    end.append(torch.nn.functional.softmax(end_scores_y, dim=1).detach().numpy()[0])

    # model, start_ind, end_ind = ensemble_confidence_prob(start, end)
    # model, start_ind, end_ind = ensemble_confidence_score(start, end)


    # print("ANSWER: %d %d %d" %(model, start_ind, end_ind))

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    # mode = {'equal', 'ranks', 'f1'}
    model, answer_start, answer_end = combine_scores2(start, end, mode="equal")
    
    # There's a chance this might hit index out of bounds
    if model == 0:
      answer = tokenizer_x.decode(input_dict_x['input_ids'][0, answer_start:answer_end+1], skip_special_tokens=True)
    else:
      answer = tokenizer_y.decode(input_dict_y['input_ids'][0, answer_start:answer_end+1], skip_special_tokens=True)
    # print('Answer: "' + answer + '"')
    return answer

#### Ensembling methods

In [0]:
def ensemble_confidence_score(start_scores, end_scores):
  prob, answer_start, answer_end, model = 0, 0, 0, 0
  for i in range(len(start_scores)):
    start = np.argmax(start_scores[i])
    end = np.argmax(end_scores[i])
    if (start_scores[i][start]*end_scores[i][end]) > prob:
      prob = start_scores[i][start]*end_scores[i][end]
      model = i 
      answer_start = start 
      answer_end = end 

  return model, answer_start, answer_end

def ensemble_confidence_prob(start_prob, end_prob):
  prob, answer_start, answer_end, model = 0.0, 0, 0, 0
  for i in range(len(start_prob)):
    # print("%f %f " %(max(start_prob[i]), max(end_prob[i])))
    for j in range(len(start_prob[i])):
      temp = start_prob[i][j] * end_prob[i]
      if max(temp) > prob:
        prob = max(temp)
        model = i
        answer_start = j
        answer_end = torch.argmax(torch.Tensor(temp))
    # print("%d %d %d %f" %(model, answer_start, answer_end, prob))

  return model, answer_start, answer_end
      

  # ans = torch.mm(torch.transpose(torch.Tensor(start_prob), 0, 1), torch.Tensor(end_prob))
  # print(ans)
  # return model, answer_start, answer_end

def ensemble_weighted_average_score(ranks, start_scores, end_scores):
  # use F1 ranks
  prob, answer_start, answer_end, model = 0.0, 0, 0, 0
  for i in range(len(start_scores)):
    start = np.argmax(start_scores[i])
    end = np.argmax(end_scores[i])
    if (ranks[i] * start_scores[i][start]*end_scores[i][end]) > prob:
      prob = ranks[i] * start_scores[i][start]*end_scores[i][end]
      model = i 
      answer_start = start 
      answer_end = end 
    # print("%d %d %d %f" %(model, answer_start, answer_end, prob))

  return model, answer_start, answer_end

def ensemble_weighted_average_prob(ranks, start_prob, end_prob):
  # use F1 ranks
  prob, answer_start, answer_end, model = 0.0, 0, 0, 0
  for i in range(len(start_prob)):
    for j in range(len(start_prob[i])):
      temp = ranks[i] * start_prob[i][j] * end_prob[i]
      if max(temp) > prob:
        prob = max(temp)
        model = i
        answer_start = j
        answer_end = torch.argmax(torch.Tensor(temp))
    # print("%d %d %d %f" %(model, answer_start, answer_end, prob))

  return model, answer_start, answer_end

In [0]:
def add_lists(tup):
  ans = tup[0]

  for i in range(1, len(tup)):
    np.add(ans, tup[i])

  return ans

def ensemble_weighted_average_new(ranks, start_prob, end_prob):
  prob, answer_start, answer_end, max_model, ans_val, max_start, max_end = [], 0, 0, 0, 0.0, 0, 0
  # number of models
  for i in range(len(start_prob)):
    # number of spans
    temp = []
    for j in range(len(start_prob[i])):
      t = ranks[i] * start_prob[i][j] * np.asarray(end_prob[i])
      temp.append(t)
      temp_val = max(t)
      if temp_val > ans_val:
        ans_val = temp_val 
        max_model = i
        max_start = j
        max_end = np.where(t == temp_val)
    
    prob.append(temp)

  # print(prob )
  answer_mat = np.asarray(prob[0])
  for i in range(1, len(prob)):
    answer_mat = np.add(answer_mat, np.asarray(prob[i]))

  # print(answer_mat)
  # print(answer_mat) 
  a = np.amax(answer_mat)
  answer_start = np.where(answer_mat == a)
  answer_end = int(answer_start[1][0])
  answer_start = int(answer_start[0][0])
    # print("%d %d %d %f" %(model, answer_start, answer_end, prob))

  return max_model, max_start, max_end, answer_start, answer_end

In [0]:
def combine_scores2(start_scores, end_scores, mode="equal"):
  if mode == "equal":
    l = len(start_scores)
    ranks = [1.0/l] * l
    model, max_start, max_end, start, end = ensemble_weighted_average_new(ranks, start_scores, end_scores)
    return model, start, end

  elif mode == "ranks":
    l = len(start_scores)
    ranks = list(range(1, l+1))
    l = l*(l+1)//2
    ranks = [x/float(l) for x in ranks]
    model, max_start, max_end, start, end = ensemble_weighted_average_new(ranks, start_scores, end_scores)
    return model, start, end

  elif mode == "f1":
    ranks = []
    model, max_start, max_end, start, end = ensemble_weighted_average_new(ranks, start_scores, end_scores)
    return model, start, end

#### Model loading

If you load new models, please do so in a new cell and comment which models are being loaded

In [0]:
"""
MINI / SMALL / MEDIUM
"""

from transformers import BertForQuestionAnswering, BertTokenizer

bert_mini_model = BertForQuestionAnswering.from_pretrained('mrm8488/bert-mini-finetuned-squadv2')
bert_mini_tokenizer = BertTokenizer.from_pretrained('mrm8488/bert-mini-finetuned-squadv2')
print("Bert Mini loaded...")

bert_small_model = BertForQuestionAnswering.from_pretrained('mrm8488/bert-small-finetuned-squadv2')
bert_small_tokenizer = BertTokenizer.from_pretrained('mrm8488/bert-small-finetuned-squadv2')
print("Bert Small loaded...")

bert_med_model = BertForQuestionAnswering.from_pretrained('mrm8488/bert-medium-finetuned-squadv2')
bert_med_tokenizer = BertTokenizer.from_pretrained('mrm8488/bert-medium-finetuned-squadv2')
print("Bert Medium loaded...")


Bert Mini loaded...
Bert Small loaded...
Bert Medium loaded...


In [0]:
"""
SMALL / MEDIUM / XLNET
"""

from transformers import BertForQuestionAnswering, BertTokenizer, XLNetForQuestionAnswering, XLNetTokenizer

bert_small_model = BertForQuestionAnswering.from_pretrained('mrm8488/bert-small-finetuned-squadv2')
bert_small_tokenizer = BertTokenizer.from_pretrained('mrm8488/bert-small-finetuned-squadv2')
print("Bert Small loaded...")

bert_med_model = BertForQuestionAnswering.from_pretrained('mrm8488/bert-medium-finetuned-squadv2')
bert_med_tokenizer = BertTokenizer.from_pretrained('mrm8488/bert-medium-finetuned-squadv2')
print("Bert Medium loaded...")

XLNET_model = XLNetForQuestionAnswering.from_pretrained('xlnet-base-cased')
XLNET_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
print("XLNET loaded...")

In [0]:
"""
SMALL / MEDIUM / DISTIL BASE
"""

from transformers import BertForQuestionAnswering, BertTokenizer, DistilBertForQuestionAnswering, DistilBertTokenizer

bert_small_model = BertForQuestionAnswering.from_pretrained('mrm8488/bert-small-finetuned-squadv2')
bert_small_tokenizer = BertTokenizer.from_pretrained('mrm8488/bert-small-finetuned-squadv2')
print("Bert Small loaded...")

bert_med_model = BertForQuestionAnswering.from_pretrained('mrm8488/bert-medium-finetuned-squadv2')
bert_med_tokenizer = BertTokenizer.from_pretrained('mrm8488/bert-medium-finetuned-squadv2')
print("Bert Medium loaded...")

distil_bert_model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-cased')
distil_bert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
print("DistilBert loaded...")

In [0]:
"""
ALBERT
"""

from transformers import AlbertTokenizer, AlbertForQuestionAnswering

albert_tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
albert_model = AlbertForQuestionAnswering.from_pretrained('albert-base-v2')

#### Answer function

In [0]:
def answer_question(question, text, alpha=.5):
  """
  Answering wrapper, change which function you use and which models you use it with
  """
  
  return ensemble_triple(bert_mini_model, bert_small_model, bert_med_model, bert_mini_tokenizer, bert_small_tokenizer, bert_med_tokenizer,
                        question, text, alpha)
  
  
  """
  return ensemble_double(bert_mini_model, bert_med_model, bert_mini_tokenizer, bert_med_tokenizer,
                        question, text, alpha)
  """

#### Tester code

Check if prediction crashes on you.

In [0]:
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

bert_abstract = "We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models (Peters et al., 2018a; Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial taskspecific architecture modifications. BERT is conceptually simple and empirically powerful. It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE score to 80.5% (7.7% point absolute improvement), MultiNLI accuracy to 86.7% (4.6% absolute improvement), SQuAD v1.1 question answering Test F1 to 93.2 (1.5 point absolute improvement) and SQuAD v2.0 Test F1 to 83.1 (5.1 point absolute improvement)."

print(wrapper.fill(bert_abstract))

We introduce a new language representation model called BERT, which stands for
Bidirectional Encoder Representations from Transformers. Unlike recent language
representation models (Peters et al., 2018a; Radford et al., 2018), BERT is
designed to pretrain deep bidirectional representations from unlabeled text by
jointly conditioning on both left and right context in all layers. As a result,
the pre-trained BERT model can be finetuned with just one additional output
layer to create state-of-the-art models for a wide range of tasks, such as
question answering and language inference, without substantial taskspecific
architecture modifications. BERT is conceptually simple and empirically
powerful. It obtains new state-of-the-art results on eleven natural language
processing tasks, including pushing the GLUE score to 80.5% (7.7% point absolute
improvement), MultiNLI accuracy to 86.7% (4.6% absolute improvement), SQuAD v1.1
question answering Test F1 to 93.2 (1.5 point absolute improvement) 

In [0]:
question = "What does the 'B' in BERT stand for?"

print(answer_question(question, bert_abstract))

bidirectional encoder representations from transformers


In [0]:
question = "What are some example applications of BERT?"

print(answer_question(question, bert_abstract))

eleven natural language processing tasks


### Prediction on SQUAD

This will tell you how many predictions have failed and will -> "" them by default.

In [0]:
#COLAB DRIVE MOUNT
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Builtin preprocessor
from transformers.data.processors.squad import SquadV2Processor

processor = SquadV2Processor()
eval_examples = processor.get_dev_examples("/content/gdrive/My Drive/squad")

100%|██████████| 35/35 [00:03<00:00,  8.83it/s]


In [0]:
# The SquadExample class fields
"""
self.qas_id = qas_id
self.question_text = question_text
self.context_text = context_text
self.answer_text = answer_text
self.title = title
self.is_impossible = is_impossible
self.answers = answers
"""

In [0]:
def run_predictions(eval_set, out_filename, n_examples=None, padding=False):
  """
  Automatic prediction script with options for checkpointing, error coverage,
  padding of incomplete experiments and filedumping.
  """
  predictions = dict()
  checkpoints = []
  error_count = 0
  for k in range(1000, 11000, 1000):
    # checkpoints.append(k)
    pass

  if not n_examples:
    for i, example in enumerate(eval_set):
      try:
        predictions[example.qas_id] = answer_question(example.question_text, example.context_text)
      except:
        predictions[example.qas_id] = ""
        error_count += 1
      output.clear('status_text')
      with output.use_tags('status_text'):
        print(f"Loaded {i+1}/{len(eval_set)} Failed predictions: {error_count}")
      if i in checkpoints:
        with open('/content/gdrive/My Drive/squad/'+str(int(i/1000))+'k_'+out_filename, 'w') as fp:
          json.dump(predictions, fp)
        print(f"Saved checkpoint {i/1000}k")
  else:
    for i in range(n_examples):
      example = eval_set[i]
      predictions[example.qas_id] = answer_question(example.question_text, example.context_text)
      output.clear('status_text')
      with output.use_tags('status_text'):
        print(f"Loaded {i+1}/{n_examples}")
        
    if padding:
      for i in range(n_examples, len(eval_set), 1):
        example = eval_set[i]
        predictions[example.qas_id] = ""
        output.clear('status_text')
        with output.use_tags('status_text'):
          print(f"Loaded {i+1}/{len(eval_set)}")

  output.clear('status_text')

  with open('/content/gdrive/My Drive/squad/'+out_filename, 'w') as fp:
      json.dump(predictions, fp)
  

In [0]:
# run_predictions(eval_examples, "mini_small_med_w2_a5_t33.json")
run_predictions(eval_examples, "mini_small_med_eq.json")
print("Done")

Done


### The eval script for reference

In [0]:
"""Official evaluation script for SQuAD version 2.0.

In addition to basic functionality, we also compute additional statistics and
plot precision-recall curves if an additional na_prob.json file is provided.
This file is expected to map question ID's to the model's predicted probability
that a question is unanswerable.
"""
import argparse
import collections
import json
import numpy as np
import os
import re
import string
import sys

OPTS = None

def parse_args():
  parser = argparse.ArgumentParser('Official evaluation script for SQuAD version 2.0.')
  parser.add_argument('data_file', metavar='data.json', help='Input data JSON file.')
  parser.add_argument('pred_file', metavar='pred.json', help='Model predictions.')
  parser.add_argument('--out-file', '-o', metavar='eval.json',
                      help='Write accuracy metrics to file (default is stdout).')
  parser.add_argument('--na-prob-file', '-n', metavar='na_prob.json',
                      help='Model estimates of probability of no answer.')
  parser.add_argument('--na-prob-thresh', '-t', type=float, default=1.0,
                      help='Predict "" if no-answer probability exceeds this (default = 1.0).')
  parser.add_argument('--out-image-dir', '-p', metavar='out_images', default=None,
                      help='Save precision-recall curves to directory.')
  parser.add_argument('--verbose', '-v', action='store_true')
  if len(sys.argv) == 1:
    parser.print_help()
    sys.exit(1)
  return parser.parse_args()

def make_qid_to_has_ans(dataset):
  qid_to_has_ans = {}
  for article in dataset:
    for p in article['paragraphs']:
      for qa in p['qas']:
        qid_to_has_ans[qa['id']] = bool(qa['answers'])
  return qid_to_has_ans

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
  return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def get_raw_scores(dataset, preds):
  exact_scores = {}
  f1_scores = {}
  for article in dataset:
    for p in article['paragraphs']:
      for qa in p['qas']:
        qid = qa['id']
        gold_answers = [a['text'] for a in qa['answers']
                        if normalize_answer(a['text'])]
        if not gold_answers:
          # For unanswerable questions, only correct answer is empty string
          gold_answers = ['']
        if qid not in preds:
          print('Missing prediction for %s' % qid)
          continue
        a_pred = preds[qid]
        # Take max over all gold answers
        exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
        f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
  return exact_scores, f1_scores

def apply_no_ans_threshold(scores, na_probs, qid_to_has_ans, na_prob_thresh):
  new_scores = {}
  for qid, s in scores.items():
    pred_na = na_probs[qid] > na_prob_thresh
    if pred_na:
      new_scores[qid] = float(not qid_to_has_ans[qid])
    else:
      new_scores[qid] = s
  return new_scores

def make_eval_dict(exact_scores, f1_scores, qid_list=None):
  if not qid_list:
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total),
        ('f1', 100.0 * sum(f1_scores.values()) / total),
        ('total', total),
    ])
  else:
    total = len(qid_list)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores[k] for k in qid_list) / total),
        ('f1', 100.0 * sum(f1_scores[k] for k in qid_list) / total),
        ('total', total),
    ])

def merge_eval(main_eval, new_eval, prefix):
  for k in new_eval:
    main_eval['%s_%s' % (prefix, k)] = new_eval[k]

def plot_pr_curve(precisions, recalls, out_image, title):
  plt.step(recalls, precisions, color='b', alpha=0.2, where='post')
  plt.fill_between(recalls, precisions, step='post', alpha=0.2, color='b')
  plt.xlabel('Recall')
  plt.ylabel('Precision')
  plt.xlim([0.0, 1.05])
  plt.ylim([0.0, 1.05])
  plt.title(title)
  plt.savefig(out_image)
  plt.clf()

def make_precision_recall_eval(scores, na_probs, num_true_pos, qid_to_has_ans,
                               out_image=None, title=None):
  qid_list = sorted(na_probs, key=lambda k: na_probs[k])
  true_pos = 0.0
  cur_p = 1.0
  cur_r = 0.0
  precisions = [1.0]
  recalls = [0.0]
  avg_prec = 0.0
  for i, qid in enumerate(qid_list):
    if qid_to_has_ans[qid]:
      true_pos += scores[qid]
    cur_p = true_pos / float(i+1)
    cur_r = true_pos / float(num_true_pos)
    if i == len(qid_list) - 1 or na_probs[qid] != na_probs[qid_list[i+1]]:
      # i.e., if we can put a threshold after this point
      avg_prec += cur_p * (cur_r - recalls[-1])
      precisions.append(cur_p)
      recalls.append(cur_r)
  if out_image:
    plot_pr_curve(precisions, recalls, out_image, title)
  return {'ap': 100.0 * avg_prec}

def run_precision_recall_analysis(main_eval, exact_raw, f1_raw, na_probs, 
                                  qid_to_has_ans, out_image_dir):
  if out_image_dir and not os.path.exists(out_image_dir):
    os.makedirs(out_image_dir)
  num_true_pos = sum(1 for v in qid_to_has_ans.values() if v)
  if num_true_pos == 0:
    return
  pr_exact = make_precision_recall_eval(
      exact_raw, na_probs, num_true_pos, qid_to_has_ans,
      out_image=os.path.join(out_image_dir, 'pr_exact.png'),
      title='Precision-Recall curve for Exact Match score')
  pr_f1 = make_precision_recall_eval(
      f1_raw, na_probs, num_true_pos, qid_to_has_ans,
      out_image=os.path.join(out_image_dir, 'pr_f1.png'),
      title='Precision-Recall curve for F1 score')
  oracle_scores = {k: float(v) for k, v in qid_to_has_ans.items()}
  pr_oracle = make_precision_recall_eval(
      oracle_scores, na_probs, num_true_pos, qid_to_has_ans,
      out_image=os.path.join(out_image_dir, 'pr_oracle.png'),
      title='Oracle Precision-Recall curve (binary task of HasAns vs. NoAns)')
  merge_eval(main_eval, pr_exact, 'pr_exact')
  merge_eval(main_eval, pr_f1, 'pr_f1')
  merge_eval(main_eval, pr_oracle, 'pr_oracle')

def histogram_na_prob(na_probs, qid_list, image_dir, name):
  if not qid_list:
    return
  x = [na_probs[k] for k in qid_list]
  weights = np.ones_like(x) / float(len(x))
  plt.hist(x, weights=weights, bins=20, range=(0.0, 1.0))
  plt.xlabel('Model probability of no-answer')
  plt.ylabel('Proportion of dataset')
  plt.title('Histogram of no-answer probability: %s' % name)
  plt.savefig(os.path.join(image_dir, 'na_prob_hist_%s.png' % name))
  plt.clf()

def find_best_thresh(preds, scores, na_probs, qid_to_has_ans):
  num_no_ans = sum(1 for k in qid_to_has_ans if not qid_to_has_ans[k])
  cur_score = num_no_ans
  best_score = cur_score
  best_thresh = 0.0
  qid_list = sorted(na_probs, key=lambda k: na_probs[k])
  for i, qid in enumerate(qid_list):
    if qid not in scores: continue
    if qid_to_has_ans[qid]:
      diff = scores[qid]
    else:
      if preds[qid]:
        diff = -1
      else:
        diff = 0
    cur_score += diff
    if cur_score > best_score:
      best_score = cur_score
      best_thresh = na_probs[qid]
  return 100.0 * best_score / len(scores), best_thresh

def find_all_best_thresh(main_eval, preds, exact_raw, f1_raw, na_probs, qid_to_has_ans):
  best_exact, exact_thresh = find_best_thresh(preds, exact_raw, na_probs, qid_to_has_ans)
  best_f1, f1_thresh = find_best_thresh(preds, f1_raw, na_probs, qid_to_has_ans)
  main_eval['best_exact'] = best_exact
  main_eval['best_exact_thresh'] = exact_thresh
  main_eval['best_f1'] = best_f1
  main_eval['best_f1_thresh'] = f1_thresh

def main():
  with open(OPTS.data_file) as f:
    dataset_json = json.load(f)
    dataset = dataset_json['data']
  with open(OPTS.pred_file) as f:
    preds = json.load(f)
  if OPTS.na_prob_file:
    with open(OPTS.na_prob_file) as f:
      na_probs = json.load(f)
  else:
    na_probs = {k: 0.0 for k in preds}
  qid_to_has_ans = make_qid_to_has_ans(dataset)  # maps qid to True/False
  has_ans_qids = [k for k, v in qid_to_has_ans.items() if v]
  no_ans_qids = [k for k, v in qid_to_has_ans.items() if not v]
  exact_raw, f1_raw = get_raw_scores(dataset, preds)
  exact_thresh = apply_no_ans_threshold(exact_raw, na_probs, qid_to_has_ans,
                                        OPTS.na_prob_thresh)
  f1_thresh = apply_no_ans_threshold(f1_raw, na_probs, qid_to_has_ans,
                                     OPTS.na_prob_thresh)
  out_eval = make_eval_dict(exact_thresh, f1_thresh)
  if has_ans_qids:
    has_ans_eval = make_eval_dict(exact_thresh, f1_thresh, qid_list=has_ans_qids)
    merge_eval(out_eval, has_ans_eval, 'HasAns')
  if no_ans_qids:
    no_ans_eval = make_eval_dict(exact_thresh, f1_thresh, qid_list=no_ans_qids)
    merge_eval(out_eval, no_ans_eval, 'NoAns')
  if OPTS.na_prob_file:
    find_all_best_thresh(out_eval, preds, exact_raw, f1_raw, na_probs, qid_to_has_ans)
  if OPTS.na_prob_file and OPTS.out_image_dir:
    run_precision_recall_analysis(out_eval, exact_raw, f1_raw, na_probs, 
                                  qid_to_has_ans, OPTS.out_image_dir)
    histogram_na_prob(na_probs, has_ans_qids, OPTS.out_image_dir, 'hasAns')
    histogram_na_prob(na_probs, no_ans_qids, OPTS.out_image_dir, 'noAns')
  if OPTS.out_file:
    with open(OPTS.out_file, 'w') as f:
      json.dump(out_eval, f)
  else:
    print(json.dumps(out_eval, indent=2))

### Code dump

Contains unused or removed code

In [0]:
    """
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids_x = tokenizer_x.encode(question, answer_text)
    input_ids_y = tokenizer_y.encode(question, answer_text)
    
    encoded_dict_x = tokenizer_x.encode_plus(text=question, text_pair=answer_text, add_special=True)
    input_ids_x = encoded_dict_x['token_type_ids']

    encoded_dict_y = tokenizer_y.encode_plus(text=question, text_pair=answer_text, add_special=True)
    input_ids_y = encoded_dict_y['token_type_ids']
    
    print(f"X: {len(input_ids_x)}, Y: {len(input_ids_y)}")

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index_x = input_ids_x.index(tokenizer_x.sep_token_id)
    sep_index_y = input_ids_y.index(tokenizer_y.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a_x = sep_index_x + 1
    num_seg_a_y = sep_index_y + 1

    # The remainder are segment B.
    num_seg_b_x = len(input_ids_x) - num_seg_a_x
    num_seg_b_y = len(input_ids_y) - num_seg_a_y

    # Construct the list of 0s and 1s.
    segment_ids_x = [0]*num_seg_a_x + [1]*num_seg_b_x
    segment_ids_y = [0]*num_seg_a_y + [1]*num_seg_b_y

    # There should be a segment_id for every input token.
    assert len(segment_ids_x) == len(input_ids_x)
    assert len(segment_ids_y) == len(input_ids_y)

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores_x, end_scores_x = model_x(torch.tensor([input_ids_x]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids_x])) # The segment IDs to differentiate question from answer_text
    
    start_scores_y, end_scores_y = model_y(torch.tensor([input_ids_y]), 
                                    token_type_ids=torch.tensor([segment_ids_y]))

    """

    
    """
    # Get the string versions of the input tokens.
    tokens = tokenizer_x.convert_ids_to_tokens(input_ids_x)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
    """


    """
    from transformers import squad_convert_examples_to_features
    # Convert to dataset
    eval_dataset_bert = squad_convert_examples_to_features(
        examples=eval_examples,
        tokenizer=bert_tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        )

    eval_dataset_albert = squad_convert_examples_to_features(
        examples=eval_examples,
        tokenizer=albert_tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        )     
    """